In [1]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode

Input data source: [Danh sách đơn vị hành chính cấp huyện của Việt Nam - Wikipedia.org](https://vi.wikipedia.org/wiki/Danh_s%C3%A1ch_%C4%91%C6%A1n_v%E1%BB%8B_h%C3%A0nh_ch%C3%ADnh_c%E1%BA%A5p_huy%E1%BB%87n_c%E1%BB%A7a_Vi%E1%BB%87t_Nam)

In [2]:
df = pd.read_csv('../data/input/wikipedia.csv')

In [3]:
df

,district,province,level
0,A Lưới,Thừa Thiên Huế,NaN
1,An Biên,Kiên Giang,NaN
2,An Dương,Hải Phòng,NaN
3,An Khê,Gia Lai,thị xã
4,An Lão,Hải Phòng,NaN
...,...,...,...
699,Yên Phong,Bắc Ninh,NaN
700,Yên Sơn,Tuyên Quang,NaN
701,Yên Thành,Nghệ An,NaN
702,Yên Thế,Bắc Giang,NaN


In [4]:
def create_province_english(text):
    u_text = unidecode(text)
    c_text = str(u_text).title().strip()
    c_text = re.sub(r'\sProvince$|^Tinh\s|^Thanh\sPho\s|\sCity$', '', c_text, flags=re.IGNORECASE)
    # if c_text == 'Ho Chi Minh':
    #     c_text = 'Ho Chi Minh City'
    return c_text

In [5]:
def create_district_english(text, level=None):
    u_text = unidecode(text)
    c_text = str(u_text).title().strip()
    
    if re.search(r'^Quan \d{1,2}', c_text, flags=re.IGNORECASE):
        c_text = re.sub(r'^Quan ', 'District ', c_text, flags=re.IGNORECASE)
    
    if isinstance(level, str):
        c_text = f'{c_text} {level}'
        
    return c_text

In [6]:
df['province'] = df['province'].str.replace('–', '-').str.replace('Thành phố ', '')
df['district'] = df['district'].str.replace('–', '-')

In [7]:
df['level'] = df['level'].str.capitalize()

In [8]:
df['level'].unique()

array([nan, 'Thị xã', 'Quận', 'Thành phố', 'Huyện đảo'], dtype=object)

In [9]:
keep_level = ['Thị xã', 'Thành phố']

In [10]:
level_english = {
    'Thị xã': 'Town',
    'Thành phố': 'City'
}

In [11]:
df['level'] = np.where(df['level'].isin(keep_level), df['level'], np.nan)

In [12]:
df['level_english'] = df['level'].map(level_english)

In [13]:
df['district_long'] = np.where(~df['level'].isna(), df['level'] + ' ' + df['district'], df['district'])

In [14]:
df['district_english'] = df['district'].apply(unidecode)

In [15]:
df['district_long_english'] = df.apply(lambda x: create_district_english(x.district, x.level_english), axis=1)

In [16]:
df['province_english'] = df['province'].apply(create_province_english)

In [17]:
df

,district,province,level,level_english,district_long,district_english,district_long_english,province_english
0,A Lưới,Thừa Thiên Huế,NaN,NaN,A Lưới,A Luoi,A Luoi,Thua Thien Hue
1,An Biên,Kiên Giang,NaN,NaN,An Biên,An Bien,An Bien,Kien Giang
2,An Dương,Hải Phòng,NaN,NaN,An Dương,An Duong,An Duong,Hai Phong
3,An Khê,Gia Lai,Thị xã,Town,Thị xã An Khê,An Khe,An Khe Town,Gia Lai
4,An Lão,Hải Phòng,NaN,NaN,An Lão,An Lao,An Lao,Hai Phong
...,...,...,...,...,...,...,...,...
699,Yên Phong,Bắc Ninh,NaN,NaN,Yên Phong,Yen Phong,Yen Phong,Bac Ninh
700,Yên Sơn,Tuyên Quang,NaN,NaN,Yên Sơn,Yen Son,Yen Son,Tuyen Quang
701,Yên Thành,Nghệ An,NaN,NaN,Yên Thành,Yen Thanh,Yen Thanh,Nghe An
702,Yên Thế,Bắc Giang,NaN,NaN,Yên Thế,Yen The,Yen The,Bac Giang


In [18]:
df.to_csv('../data/output/district_administrative_unit_of_vietnam.csv', index=False, encoding='utf-8')